In [1]:
import json
from pathlib import Path
from collections import Counter, defaultdict
from itertools import chain
from urllib.parse import quote

from lxml import etree
from SPARQLWrapper import SPARQLWrapper
import requests
from tqdm.notebook import tqdm

## 北海道の市町村TopoJSON

[北海道のTopoJSON: 札幌市の区をマージ / Sorami Hisamoto / Observable](https://observablehq.com/@sorami/hokkaido-topojson-merge-sapporo)

In [2]:
with open("../public/hokkaido.topojson") as fp:
    topo = json.load(fp)
len(topo["objects"]["hokkaido"]["geometries"])

185

In [3]:
topo["objects"]["hokkaido"]["geometries"][0]["properties"]

{'code': '01100', 'name': '札幌市', 'region': '石狩振興局'}

In [4]:
topo_city_list = [
    d["properties"] for d in topo["objects"]["hokkaido"]["geometries"]
]
topo_city_list[:5]

[{'code': '01100', 'name': '札幌市', 'region': '石狩振興局'},
 {'code': '01202', 'name': '函館市', 'region': '渡島総合振興局'},
 {'code': '01203', 'name': '小樽市', 'region': '後志総合振興局'},
 {'code': '01205', 'name': '室蘭市', 'region': '胆振総合振興局'},
 {'code': '01331', 'name': '松前町', 'region': '渡島総合振興局'}]

In [5]:
topo_city_names = [
    d["name"] for d in topo_city_list
]
topo_city_names[:5]

['札幌市', '函館市', '小樽市', '室蘭市', '松前町']

In [6]:
Counter([d[-1] for d in topo_city_names])

Counter({'市': 35, '町': 129, '村': 21})

### 市町村名に使われる文字

In [7]:
count_per_char = Counter(chain.from_iterable(topo_city_names))
len(count_per_char)

243

In [8]:
count_per_char.most_common(10)

[('町', 129),
 ('市', 36),
 ('別', 22),
 ('村', 21),
 ('川', 11),
 ('幌', 9),
 ('内', 9),
 ('富', 6),
 ('部', 6),
 ('上', 6)]

### 重複する市町村名

In [9]:
Counter(topo_city_names).most_common(2)

[('泊村', 2), ('札幌市', 1)]

In [10]:
[d for d in topo_city_list if d["name"] == "泊村"]

[{'code': '01403', 'name': '泊村', 'region': '後志総合振興局'},
 {'code': '01696', 'name': '泊村', 'region': '根室振興局'}]

### 市町村名の文字数

In [11]:
sorted(Counter([len(d) for d in topo_city_names]).items())

[(2, 3), (3, 136), (4, 39), (5, 7)]

In [12]:
sorted(topo_city_names, key=lambda d: len(d), reverse=True)[:10]

['新十津川町',
 '上富良野町',
 '中富良野町',
 '南富良野町',
 '音威子府村',
 '利尻富士町',
 '新ひだか町',
 'せたな町',
 '岩見沢市',
 '苫小牧市']

## 市町村情報の集約

### 国土数値情報の市区町村役場データをパース

元データ: [国土数値情報 | 市区町村役場データ](https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-P34.html)

```xml
<ksj:Dataset gml:id="P34Dataset" xmlns:ksj="http://nlftp.mlit.go.jp/ksj/schemas/ksj-app" xmlns:gml="http://www.opengis.net/gml/3.2" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://nlftp.mlit.go.jp/ksj/schemas/ksj-app KsjAppSchema-P34-v2_0.xsd">

...
    
<gml:Point gml:id="p1">
	<gml:pos>43.06197200 141.35437400</gml:pos>
</gml:Point>

...

<ksj:LocalGovernmentOfficeAndPublicMeetingFacility gml:id="fe01_1">
	<ksj:position xlink:href="#p1"/>
	<ksj:administrativeAreaCode codeSpace="AdministrativeAreaCode.xml">01100</ksj:administrativeAreaCode>
	<ksj:publicOfficeClassification>1</ksj:publicOfficeClassification>
	<ksj:publicOfficeName>札幌市役所</ksj:publicOfficeName>
	<ksj:address>札幌市中央区北1条西2</ksj:address>
</ksj:LocalGovernmentOfficeAndPublicMeetingFacility>
```

In [13]:
tree = etree.parse("./P34-14_01_GML/P34-14_01.xml")
namespaces = {
    "ksj": "http://nlftp.mlit.go.jp/ksj/schemas/ksj-app",
    "gml": "http://www.opengis.net/gml/3.2",
    "xlink": "http://www.w3.org/1999/xlink"
}

In [14]:
facility_nodes = tree.xpath("ksj:LocalGovernmentOfficeAndPublicMeetingFacility", namespaces=namespaces)
len(facility_nodes)

373

In [15]:
def parse_facility_info(node):
    href = node.xpath("ksj:position/@xlink:href", namespaces=namespaces)[0]

    area_code = node.xpath("ksj:administrativeAreaCode", namespaces=namespaces)[0].text
    
    # 施設分類: 1 = 本庁（市役所、区役所、町役場、村役場）, 2 = 支所、出張所、連絡所
    classification = node.xpath("ksj:publicOfficeClassification", namespaces=namespaces)[0].text

    name = node.xpath("ksj:publicOfficeName", namespaces=namespaces)[0].text
    address = node.xpath("ksj:address", namespaces=namespaces)[0].text
    
    return {
        "href": href,
        "code": area_code,
        "class": classification,
        "name": name,
        "address": address
    }

In [16]:
facilities = [parse_facility_info(node) for node in facility_nodes]
facilities[:3]

[{'href': '#p1',
  'code': '01100',
  'class': '1',
  'name': '札幌市役所',
  'address': '札幌市中央区北1条西2'},
 {'href': '#p2',
  'code': '01101',
  'class': '1',
  'name': '札幌市中央区役所',
  'address': '札幌市中央区南3条西11'},
 {'href': '#p3',
  'code': '01102',
  'class': '1',
  'name': '札幌市北区役所',
  'address': '札幌市北区北24条西6-1-1'}]

### 緯度経度

In [17]:
facility_positions = {}

for fac in facilities:
    if fac["class"] != "1":
        continue
    assert fac["code"] not in facility_positions, fac["code"]

    href = fac["href"].replace("#", "")
    pos_raw = tree.xpath(f"gml:Point[@gml:id='{href}']/gml:pos", namespaces=namespaces)[0].text
    lat, lon = map(float, pos_raw.split())
    
    facility_positions[fac["code"]] = {"lat": lat, "lon": lon}

### 市町村

In [18]:
municipalities = {}
northern_territories = {}

for city in topo_city_list:
    code = city["code"]
    if code not in facility_positions:
        northern_territories[code] = {
            "code": code,
            "name": city["name"],
            "region": city["region"]
        }
        continue
    
    pos = facility_positions[code]
    assert code not in municipalities
    municipalities[code] = {
        "code": code,
        "name": city["name"],
        "region": city["region"],
        "coordinates": [pos["lon"], pos["lat"]]
    }

len(municipalities), list(municipalities.items())[:3]

(179,
 [('01100',
   {'code': '01100',
    'name': '札幌市',
    'region': '石狩振興局',
    'coordinates': [141.354374, 43.061972]}),
  ('01202',
   {'code': '01202',
    'name': '函館市',
    'region': '渡島総合振興局',
    'coordinates': [140.729108, 41.768712]}),
  ('01203',
   {'code': '01203',
    'name': '小樽市',
    'region': '後志総合振興局',
    'coordinates': [140.99460538, 43.19075267]})])

### 北方地域: 緯度経度情報を追加

In [19]:
northern_territories

{'01700': {'code': '01700', 'name': '蘂取村', 'region': '根室振興局'},
 '01695': {'code': '01695', 'name': '色丹村', 'region': '根室振興局'},
 '01696': {'code': '01696', 'name': '泊村', 'region': '根室振興局'},
 '01697': {'code': '01697', 'name': '留夜別村', 'region': '根室振興局'},
 '01698': {'code': '01698', 'name': '留別村', 'region': '根室振興局'},
 '01699': {'code': '01699', 'name': '紗那村', 'region': '根室振興局'}}


[歴史的行政区域データセットβ版 | Geoshapeリポジトリ](https://geoshape.ex.nii.ac.jp/city/) CC BY 4.0

代表点（ポリゴンの重心）

- [北海道色丹郡色丹村 (01695A1968) | 歴史的行政区域データセットβ版](https://geoshape.ex.nii.ac.jp/city/resource/01695A1968.html) ` 43.800739, 146.738866`
- [北海道国後郡泊村 (01696A1968) | 歴史的行政区域データセットβ版](https://geoshape.ex.nii.ac.jp/city/resource/01696A1968.html) `43.909815, 145.623708`
- [北海道国後郡留夜別村 (01697A1968) | 歴史的行政区域データセットβ版](https://geoshape.ex.nii.ac.jp/city/resource/01697A1968.html) `44.319813, 146.127440`
- [北海道択捉郡留別村 (01698A1968) | 歴史的行政区域データセットβ版](https://geoshape.ex.nii.ac.jp/city/resource/01698A1968.html) `44.830710, 147.407788`
- [北海道紗那郡紗那村 (01699A1968) | 歴史的行政区域データセットβ版](https://geoshape.ex.nii.ac.jp/city/resource/01699A1968.html) `45.199148, 148.000166`
- [北海道蘂取郡蘂取村 (01700A1968) | 歴史的行政区域データセットβ版](https://geoshape.ex.nii.ac.jp/city/resource/01700A1968.html) `45.556628, 148.772557`

In [20]:
northern_territories["01695"]["coordinates"] = [146.738866, 43.800739] # 色丹村
northern_territories["01696"]["coordinates"] = [145.623708, 43.909815] # 泊村
northern_territories["01697"]["coordinates"] = [146.127440, 44.319813] # 留夜別村
northern_territories["01698"]["coordinates"] = [147.407788, 44.830710] # 留別村
northern_territories["01699"]["coordinates"] = [148.000166, 45.199148] # 紗那村
northern_territories["01700"]["coordinates"] = [148.772557, 45.556628] # 蘂取村

In [21]:
city_dict = municipalities | northern_territories
len(city_dict)

185

### 重複する市町村名を調整

In [22]:
city_dict["01403"]["name"] = "泊村:後志"
city_dict["01696"]["name"] = "泊村:根室"

### Wikipediaページ

In [23]:
# wiki_unknown_codes = []

# for code, info in tqdm(city_dict.items()):
#     name = info["name"]
#     url = f"https://ja.wikipedia.org/wiki/{quote(name)}"
#     res = requests.get(url)
#     if res.status_code != 200:
#          wiki_unknown_codes.append(code)
        
# len(wiki_unknown_codes) # ['01403', '01696']

In [24]:
for info in city_dict.values():
    name = info["name"]
    url = f"https://ja.wikipedia.org/wiki/{quote(name)}"
    info["wikipedia"] = url

city_dict["01403"]["wikipedia"] = f"https://ja.wikipedia.org/wiki/{quote('泊村')}"
city_dict["01696"]["wikipedia"] = f"https://ja.wikipedia.org/wiki/{quote('泊村_(北海道根室振興局)')}"

### Wikipedia概要文

[people-map-japan/4_people_for_map.ipynb at main · sorami/people-map-japan](https://github.com/sorami/people-map-japan/blob/main/preparation/code/4_people_for_map.ipynb)

In [25]:
sparql = SPARQLWrapper(endpoint="http://ja.dbpedia.org/sparql", returnFormat="json")

def get_wikipedia_abstract(page):
    sparql.setQuery(f"""
        SELECT DISTINCT * 
        WHERE {{
            <http://ja.dbpedia.org/resource/{page}>
            dbo:abstract
            ?o .
            }}
    """)
    res = sparql.query().convert()
    bindings = res["results"]["bindings"]
    if len(bindings) < 1:
        return ""
    return bindings[0]["o"]["value"]

In [26]:
get_wikipedia_abstract("札幌市")

'札幌市（さっぽろし）は、北海道の道央地方に位置し、石狩振興局に属する市。道庁所在地にして石狩振興局所在地でもあり、北海道の政治・経済・文化の中心地となっている。北海道最大の人口（約196万人）を有する政令指定都市である。'

In [27]:
for info in tqdm(city_dict.values()):
    info["abstract"] = get_wikipedia_abstract(info["name"])

  0%|          | 0/185 [00:00<?, ?it/s]

In [28]:
for code, info in city_dict.items():
    if not info["abstract"]:
        print(code, info["name"])

01403 泊村:後志
01696 泊村:根室


In [29]:
get_wikipedia_abstract("泊村")

'泊村（とまりむら）は、北海道後志総合振興局管内の古宇郡に属する村。北海道で唯一の原子力発電所があるため、村の財政は豊かである。 村名の由来は、中心地の入り江を指す、アイヌ語の「モイレ・トマリ（静かな・泊地）」である。'

In [30]:
get_wikipedia_abstract("泊村_(北海道根室振興局)")

'泊村（とまりむら）は、北海道根室振興局国後郡の国後島の西半分を占める村。2021年現在泊村を含む北方領土に日本の施政権は及んでおらず、法令上のみ存在する村となっている。 当該地域の領有権に関する詳細は千島列島及び北方領土問題の項目を、現状に関してはサハリン州#南クリル管区の項目を参照のこと。'

In [31]:
city_dict["01403"]["abstract"] = get_wikipedia_abstract("泊村")
city_dict["01696"]["abstract"] = get_wikipedia_abstract("泊村_(北海道根室振興局)")

assert all([d["abstract"] for d in city_dict.values()])

# 保存

In [32]:
cities = list(city_dict.values())

In [33]:
with open("../public/cities.json", "w") as fp:
    json.dump(cities, fp, ensure_ascii=False, indent=2)

In [34]:
!ls -lh ../public/

total 528
-rw-r--r--  1 sorami  staff    94K  9  3 20:20 cities.json
-rw-r--r--@ 1 sorami  staff   126K  9  3 15:51 hokkaido.topojson


In [35]:
!head ../public/cities.json

[
  {
    "code": "01100",
    "name": "札幌市",
    "region": "石狩振興局",
    "coordinates": [
      141.354374,
      43.061972
    ],
    "wikipedia": "https://ja.wikipedia.org/wiki/%E6%9C%AD%E5%B9%8C%E5%B8%82",
